<a href="https://colab.research.google.com/github/mlindsey19/EvolComp-Rucksack/blob/master/Rucksack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Knapsack Problem

In [0]:
import numpy as np

float_formatter = lambda x: "%.2f" % x
float_formatter1 = lambda x: "%.1f" % x
float_formatter2 = lambda x:  str(x)[0] if ( 0 <= x ) else str(x)[0:2] #for viewing aligned columns. ONLY SHOWS FIRST DIGIT
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

L = 20
N = 10

bits = np.random.randint(low=0,high=2,size=20)
def new_bits():
  global bits
  bits = np.random.randint(low=0,high=2,size=20)
  return bits # need this return
  
vals = np.random.normal(loc=5.0, scale=4.0, size=20)
wts = np.random.normal(loc=5.0, scale=5.0, size=20)
ratio = vals/wts
cons = np.stack( [ vals, wts, ratio ] )

avg_c = np.sum(wts)/2
restrict_c = np.amax(wts) * 2
D=0
CAP = avg_c

def new_constraits():
  global avg_c,restict_c, vals, wts, cons
  vals = np.random.normal(loc=20.0, scale=6.0, size=20)
  wts = np.random.normal(loc=1.0, scale=3, size=20)
  #vals += 10
  wts += 10
  ratio = vals/wts
  cons = np.stack( [ vals, wts, ratio ] )
  avg_c = np.sum(wts)/2
  restrict_c = np.amax(wts) * 2

pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
pop = np.stack(pop)

def new_pop():
  global pop
  pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
  pop = np.stack(pop)
  




In [0]:
def str_wt( x ):
  return ( x[0] * x[2] ).sum()
def str_val( x ):
  return ( x[0] * x[1] ).sum()


#Penalty
* **CAP** -> capacity 
* **D** -> degree of penalty ( zero=-1;  logarithmic = 0;  linear = 1; quadratic = 2 )





In [0]:
pen = ratio.max()
def set_pen():
  global pen
  pen = ratio.max()


def penalty(sw):
  if( sw > CAP ):
    global pen
    p = ( sw - CAP ) * pen
    p = np.log2( 1 + p) if ( D == 0 ) else p #assumes only positive values of p
    p = p*p if (D == 2) else p
    return 0 if ( D == -1 ) else p
  else:
    return 0
def new_problem():
  new_constraits()
  new_pop()
  set_pen()

#Repair


In [0]:
def repair():
  global pop
  for i in range(10):
    if ( str_wt(pop[i]) > CAP ):
      while( True ):
        while ( True ):
          k = np.random.randint(0, high = 20)
          if ( pop[i,0, k ] == 1 ):
            pop[ i ,0, k ] = 0
            break
        if ( str_wt( pop[i] ) <= CAP or ( not pop[i,0].sum() ) ):
          break

#Selection



In [0]:
new_problem()

In [0]:
def fit_vals():
  global pop
  y = np.stack( [ str_val(pop[i]) for i in range(N)])
  w = [str_wt(pop[ i ]) for i in range(N)]
  f = [ penalty(i ) for i in w ]
  av = y - f
  return np.stack(av)
def fit_wts():
  global pop
  y =  [ str_wt(pop[i]) for i in range(N)]
  return np.stack(y)

def sort_idx():
  return fit_vals().argsort()

In [0]:
linRank = np.empty(N)
def set_linRank(min):
  global linRank
  max = 2 - min
  Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
  PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
  for i in range(N):
    if (i == 0):
      linRank[ i ] = PlrMap[ i ]
    else:
      linRank[ i ] = PlrMap[ i ] + linRank[ i - 1 ]
set_linRank(0.0)

In [0]:
def selectionLR():
  global pop
  newPop = np.empty( (N, 4, L ) )
  si = sort_idx()
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx, c in enumerate(linRank):
      if( s <= c ):
        newPop[ i ] = pop[ si[ idx ] ]
        break
  pop = newPop

In [0]:
Psel = 0.7
def selectionBT():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  for i in range(N):
    s1 = np.random.randint(0,high=10)    
    s2 = np.random.randint(0,high=10)
    b = np.take( pop[:,0,:], s1, axis=0 )
    a = np.take( pop[:,0,:], s2, axis=0 )
    fit = fit_vals()
    p = np.random.uniform( 0, 1 )
    if ( fit[s1] > fit[s2] and p < Psel ):
      newPop[i] = pop[s1]
    else:
      newPop[i] = pop[s2]
  pop = newPop

#Crossover

In [0]:
Pcross = 0.6

def crossover_2pt():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  np.random.shuffle(pop)
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      j = 2 * i
      pt1 = np.random.randint(0,high=20) 
      pt2 = np.random.randint(pt1,high=20)
      pop[ j, 0, pt1:pt2 ], pop[ j + 1, 0, pt1:pt2 ] = pop[ j + 1, 0, pt1:pt2 ].copy(), pop[ j, 0, pt1:pt2 ].copy()
      
def crossover_1pt():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  np.random.shuffle(pop)
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      j = 2 * i
      pt1 = np.random.randint(0,high=20) 
      pop[ j, 0, pt1: ], pop[ j + 1, 0, :pt1 ] = pop[ j + 1, 0, pt1: ].copy(), pop[ j, 0, :pt1 ].copy()

def crossover_and():
  global pop
  np.random.shuffle(pop)
  newPop = np.empty( ( N, 4, L ) )
  for i in range( N ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      r = np.random.randint(0,high=10)
      b = np.take( pop[:,0,:], i, axis=0 )
      a = np.take( pop[:,0,:], r, axis=0 )
      newPop[i,0] = ( a.astype(bool) & b.astype(bool) ).astype(int)
    else:
      newPop[i,0] = pop[i,0]


#Mutation

In [0]:
Pmut = 0.01
def mutation():
  mutMatrix = np.random.random_sample((10,20))
  mm = (mutMatrix < Pmut).astype(int)
  pop[:,0,:] = (pop[:,0,:] + mm) % 2

#Evolution

In [129]:
new_problem()
CAP = avg_c
D=-1
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

print(fit_vals())
D=0

print(fit_vals())
D=1

print(fit_vals())

print(np.array([CAP]))

print(fit_wts())
D=2

for _ in range(100):
  selectionLR()
  crossover_2pt()
  mutation()
# D=1
# for _ in range(100):
#   selectionBT()
#   crossover_2pt()
#   mutation()
# D=2
# for _ in range(30):
#   selectionBT()
#   crossover_and()
#   mutation()
# D=2
# for _ in range(30):
#   selectionBT()
#   crossover_and()
#   mutation()
#   repair()

D=2
print("val",fit_vals())
print(np.array([CAP]))
print("wts",fit_wts())

np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )

print( pop[:,0] )

[196.06 189.21 184.42 185.86 172.10 159.83 112.95 186.07 252.29 182.04]
[187.78 181.70 177.62 177.04 172.10 156.27 112.95 177.64 241.44 182.04]
[-114.23 7.93 73.50 -267.39 172.10 149.07 112.95 -158.77 -1590.24 182.04]
[112.49]
[122.31 118.22 116.00 126.83 98.94 112.83 77.71 123.40 170.80 107.08]
val [207.51 207.51 188.37 190.18 183.01 -54450.30 207.51 207.51 207.51 207.51]
[112.49]
wts [108.42 108.42 93.58 96.62 98.83 119.89 108.42 108.42 108.42 108.42]
[[1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1]
 [1 0 0 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1]
 [1 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1]]


In [130]:
new_problem()
CAP = avg_c/8
D=-1
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )



for _ in range(30):
  selectionBT()
  crossover_1pt()
  mutation()
D=1
for _ in range(30):
  selectionBT()
  crossover_2pt()
  mutation()
D=2
for _ in range(20):
  selectionBT()
  crossover_and()
  mutation()
D=2
repair()
for _ in range(10):
  selectionLR()
  crossover_2pt()
  mutation()
 # repair()

D=2
print("val",fit_vals())
print(np.array([CAP]))
print("wts",fit_wts())

np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )

print(pop[:,0])


val [-3681.77 25.56 25.56 25.56 25.56 25.56 -15048.06 -15048.06 25.56 25.56]
[12.42]
wts [14.35 7.06 7.06 7.06 7.06 7.06 16.31 16.31 7.06 7.06]
[[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]


#Results

change capacity, penalty, add repair

In [0]:
crossList = [ crossover_2pt, crossover_1pt, crossover_and ]
selectList = [ selectionBT , selectionLR ]
import heapq
best = [[0],0]
bestList = np.empty((3,2), dtype=list)
avgvect = np.zeros(10)
avg = np.zeros((3,2))
stdv = np.zeros((3,2))


new_problem()
#CAP = restrict_c
CAP = avg_c
D=1 # penalty   zero=-1; logarithmic = 0; linear = 1; quadratic = 2 

for c, cross in enumerate(crossList):
  for s, sel in enumerate(selectList):
    for i in range(10):
      new_pop()
      best = [[0],0]
      D=0
      for _ in range(50):
        sel()
        cross()
        mutation()
      D=1
      for _ in range(50):
        sel()
        cross()
        mutation()
      D=2
      for _ in range(50):
        sel()
        cross()
        mutation()
      #repair()  # uncomment to add repair
      D=0 #this just gives a better representation of the averages
      fitval = fit_vals()
      avgvect[i] = np.mean(fitval)
      tmpbest =heapq.nlargest(1,range(10),vals.take)
      if( fitval[tmpbest] > best[1]):
        best[0] = pop[tmpbest,0]
        best[1] = fitval[tmpbest,]
    bestList[c,s] = best
    avg[c,s] = np.mean(avgvect)
    stdv[c,s] = np.std(avgvect)


np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )


# 2pt cross and binary tournament

In [141]:
cc =0
ss =0

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [231.30]
StDev: [14.99]
Best: [251.43]
Objects:
 [1 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 0 1 0 1]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


# 2pt cross and Linear Ranking

In [142]:
cc = 0
ss =1

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [220.02]
StDev: [12.32]
Best: [228.55]
Objects:
 [[0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 1 1 1 0 1]]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


# 1 pt crossover and binary tournament





In [143]:
cc = 1
ss =0

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [220.28]
StDev: [14.48]
Best: [215.59]
Objects:
 [0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


# 1 pt crossover and linear rank

In [144]:
cc = 1
ss =1

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [220.67]
StDev: [16.14]
Best: [225.16]
Objects:
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 0 0]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


# & corssover and binary tournament

In [145]:
cc = 2
ss =0

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [215.50]
StDev: [18.84]
Best: [167.60]
Objects:
 [0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


#& crossover and linear rank

In [146]:
cc = 2
ss =1

np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg[cc,ss]]))
print('StDev:',np.array([stdv[cc,ss]]))
print('Best:',bestList[cc,ss][1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[cc,ss][0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [224.44]
StDev: [16.72]
Best: [225.60]
Objects:
 [1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1]
Values:
 [22.6 17.0 20.4 20.6 25.9 5.8 16.4 22.0 25.4 18.7 25.5 10.9 24.2 18.2 31.0
 20.5 13.2 23.0 15.0 26.1]
Weights
 [10.6 13.8 10.1 15.9 10.3 8.9 8.0 9.5 10.0 12.5 16.1 16.7 11.0 9.5 11.7
 17.3 8.8 9.3 12.8 7.2]
Capacity: [115.01]


#penalty, capcity, repair


In [147]:
best = [[0],0]
bestList = np.empty((1), dtype=list)
avgvect = np.zeros(10)
avg = np.zeros(1)
stdv = np.zeros(1)


new_problem()
CAP = restrict_c
#CAP = avg_c
D=2 # penalty   zero=-1; logarithmic = 0; linear = 1; quadratic = 2 

for i in range(10):
  new_pop()
  best = [[],0]
  for _ in range(500):
    selectionLR()
    crossover_2pt()
    mutation()
  repair()  # uncomment to add repair
  fitval = fit_vals()
  avgvect[i] = np.mean(fitval)
  tmpbest =heapq.nlargest(1,range(10),vals.take)
  if( fitval[tmpbest] > best[1]):
    best[0] = pop[tmpbest,0]
    best[1] = fitval[tmpbest,]
bestList = best
avg = np.mean(avgvect)
stdv = np.std(avgvect)


np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Average:',np.array([avg]))
print('StDev:',np.array([stdv]))
print('Best:',bestList[1])
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )
print('Objects:\n',bestList[0][0])
np.set_printoptions( formatter = { 'float_kind' : float_formatter1 } )

print('Values:\n',vals)
print('Weights\n',wts)
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )
print('Capacity:',np.array([CAP]))

Average: [59.60]
StDev: [3.68]
Best: [65.29]
Objects:
 [0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0]
Values:
 [20.9 13.4 20.9 16.8 19.1 23.1 21.4 13.6 14.5 16.5 24.1 18.0 25.9 19.4
 23.0 14.7 19.7 15.6 14.9 16.2]
Weights
 [11.2 12.8 7.7 9.8 10.4 11.9 13.5 10.7 7.7 10.1 17.3 8.4 14.3 9.8 5.1 6.9
 10.1 7.8 11.5 10.2]
Capacity: [26.55]
